In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用Vertex AI TensorBoard分析器在使用预构建容器的自定义训练中训练性能分析模型

用Colab打开
用Colab Enterprise打开
在GitHub上查看
在Vertex AI Workbench中打开

## 概述

TensorFlow Profiler是一个强大的工具，可以帮助您诊断和调试性能瓶颈，并使您的模型训练更快。本教程演示了如何在使用预构建容器进行自定义训练时，在Vertex AI中启用TensorBoard Profiler。

了解有关[Vertex AI TensorBoard Profiler](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-profiler)的更多信息。

### 目标

在本教程中，您将学习如何在Vertex AI中为自定义训练作业启用TensorBoard Profiler，并使用预构建的容器。

本教程使用以下谷歌云AI服务：

- Vertex AI 训练
- Vertex AI TensorBoard

执行的步骤包括：

- 准备您的自定义训练代码，并将训练代码加载为Python软件包到预构建的容器中
- 创建并运行一个启用TensorBoard Profiler的自定义训练作业
- 查看TensorBoard Profiler仪表板，以调试您的模型训练性能

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)的[mnist数据集](https://www.tensorflow.org/datasets/catalog/mnist)。

成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
定价](https://cloud.google.com/storage/pricing)，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您预期的使用情况生成成本估算。

## 安装

安装以下所需的包以执行此笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

### 仅限Colab使用：取消注释下面的单元格以重启内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的谷歌云项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额度，可用于支付计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您正在本地运行此笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参阅支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type:"string"}

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动验证。请按照以下相关说明进行操作。

**1. Vertex AI工作台**
* 无需操作，您已经通过验证。

**2. 本地JupyterLab实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 设置服务帐号和权限

将使用服务帐号来创建自定义培训作业。如果您不想使用项目的计算引擎服务帐号，请将 SERVICE_ACCOUNT 设置为另一个服务帐号ID。您可以按照[说明](https://cloud.google.com/iam/docs/creating-managing-service-accounts#creating)创建服务帐号。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

In [ ]:
# Grant Cloud Storage permission.
! gcloud projects add-iam-policy-binding $PROJECT_ID \
        --member="serviceAccount:$SERVICE_ACCOUNT" \
        --role="roles/storage.admin" \
        --quiet

# Grant AI Platform permission.
! gcloud projects add-iam-policy-binding $PROJECT_ID \
        --member="serviceAccount:$SERVICE_ACCOUNT" \
        --role="roles/aiplatform.user" \
        --quiet

! gcloud projects get-iam-policy $PROJECT_ID \
        --filter=bindings.members:serviceAccount:$SERVICE_ACCOUNT

创建一个云存储存储桶

创建一个存储桶用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 导入库

In [ ]:
import os

from google.cloud import aiplatform

### 初始化 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

创建一个TensorBoard实例

在可视化实验之前，必须先创建一个Vertex AI TensorBoard实例，它是一个区域化资源，用于存储您的Vertex AI TensorBoard实验。您可以在一个项目中创建多个实例。您可以使用命令`gcloud ai tensorboards list`来获取您现有的TensorBoard实例列表。

设置您的TensorBoard实例显示名称

In [ ]:
TENSORBOARD_NAME = "your-tensorboard-unique"  # @param {type:"string"}

创建一个TensorBoard实例

如果您没有TensorBoard实例，请通过运行以下单元格来创建一个：

In [ ]:
tensorboard = aiplatform.Tensorboard.create(
    display_name=TENSORBOARD_NAME, project=PROJECT_ID, location=REGION
)

TENSORBOARD_INSTANCE_NAME = tensorboard.resource_name
print("TensorBoard instance name:", TENSORBOARD_INSTANCE_NAME)

## 训练一个模型

使用您自定义的训练代码来训练一个模型，选择以下其中一种选项：

- **预构建容器**：将您的自定义训练代码加载为一个 Python 包到来自 Google Cloud 的预构建容器镜像中。

- **自定义容器**：创建一个包含您自定义训练代码的自己的容器镜像。

在本教程中，您将使用预构建容器来训练一个自定义模型。

### 检查培训包

#### 包装布局

在开始培训之前，让我们看一下如何为自定义培训作业组装 Python 包。解压后，包含以下内容：

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到 Docker 镜像的操作环境的说明。

In [ ]:
PYTHON_PACKAGE_APPLICATION_DIR = "app"

source_package_file_name = f"{PYTHON_PACKAGE_APPLICATION_DIR}/dist/trainer-0.1.tar.gz"
python_package_gcs_uri = f"{BUCKET_URI}/trainer-0.1.tar.gz"

# Make folder for Python training script
! rm -rf {PYTHON_PACKAGE_APPLICATION_DIR}
! mkdir {PYTHON_PACKAGE_APPLICATION_DIR}

# Add package information
! touch {PYTHON_PACKAGE_APPLICATION_DIR}/README.md

# Make the training subfolder
! mkdir {PYTHON_PACKAGE_APPLICATION_DIR}/trainer
! touch {PYTHON_PACKAGE_APPLICATION_DIR}/trainer/__init__.py

In [ ]:
%%writefile ./{PYTHON_PACKAGE_APPLICATION_DIR}/setup.py

from setuptools import find_packages
from setuptools import setup
import setuptools

from distutils.command.build import build as _build
import subprocess

REQUIRED_PACKAGES = [
    'google-cloud-aiplatform[cloud_profiler]>=1.20.0',
    'protobuf==3.20.2',
]

setup(
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    name='trainer',
    version='0.1',
    url="wwww.google.com",
    description='Vertex AI | Training | Python Package'
)

#### 准备训练脚本

文件 `trainer/task.py` 是执行自定义训练作业的Python脚本。

您的训练代码必须配置为将TensorBoard日志写入一个Cloud Storage存储桶，其位置Vertex AI Training会自动通过预定义的环境变量 `AIP_TENSORBOARD_LOG_DIR` 提供。通常可以通过将 `os.environ['AIP_TENSORBOARD_LOG_DIR']` 提供给开源TensorBoard日志写入API来完成。例如，在TensorFlow 2.x中，您可以使用以下代码创建一个 `tensorboard_callback`：

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'],
      histogram_freq=1)
`AIP_TENSORBOARD_LOG_DIR` 是您在创建自定义训练作业时提供的 `BASE_OUTPUT_DIR` 中的一部分。

要为您的训练作业启用Vertex AI TensorBoard Profiler，请将以下内容添加到您的训练脚本中：

在顶层导入中添加以下云分析器导入：

    from google.cloud.aiplatform.training_utils import cloud_profiler

通过添加以下内容初始化云分析器插件：

    cloud_profiler.init()

In [ ]:
%%writefile ./{PYTHON_PACKAGE_APPLICATION_DIR}/trainer/task.py

import tensorflow as tf
import argparse
import os
import sys, traceback
from google.cloud.aiplatform.training_utils import cloud_profiler

"""Train an mnist model and use cloud_profiler for profiling."""

def _create_model():
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10),
        ]
    )
    return model


def main(args):
    print('Initialize the profiler ...')
    cloud_profiler.init()
    print('The profiler initiated.')

    print('Loading and preprocessing data ...')
    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    print('Creating and training model ...')

    model = _create_model()
    model.compile(
      optimizer="adam",
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      metrics=["accuracy"],
    )

    log_dir = "logs"
    if 'AIP_TENSORBOARD_LOG_DIR' in os.environ:
      log_dir = os.environ['AIP_TENSORBOARD_LOG_DIR']

    print('Setting up the TensorBoard callback ...')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1)

    print('Training model ...')
    model.fit(
        x_train,
        y_train,
        epochs=args.epochs,
        verbose=0,
        callbacks=[tensorboard_callback],
    )
    print('Training completed.')

    print('Saving model ...')

    model_dir = "model"
    if 'AIP_MODEL_DIR' in os.environ:
      model_dir = os.environ['AIP_MODEL_DIR']
    tf.saved_model.save(model, model_dir)

    print('Model saved at ' + model_dir)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--epochs", type=int, default=100, help="Number of epochs to run model."
    )

    args = parser.parse_args()
    main(args)

创建一个源码分发

您可以使用培训应用程序创建一个源码分发，并将其上传到您的云存储桶中。

In [ ]:
!cd {PYTHON_PACKAGE_APPLICATION_DIR} && python3 setup.py sdist --formats=gztar

!gsutil cp {source_package_file_name} {python_package_gcs_uri}

!gsutil ls -l {python_package_gcs_uri}

创建并运行自定义训练作业

使用打包为Python源代码分发的训练代码的[预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)映像配置一个[自定义作业](https://cloud.google.com/vertex-ai/docs/training/create-custom-job)。

In [ ]:
JOB_NAME = "tensorboard-job-unique"
MACHINE_TYPE = "n1-standard-4"
TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-9:latest"
base_output_dir = f"{BUCKET_URI}/{JOB_NAME}"
python_module_name = "trainer.task"

EPOCHS = 20
training_args = [
    "--epochs=" + str(EPOCHS),
]

In [ ]:
job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=JOB_NAME,
    python_package_gcs_uri=python_package_gcs_uri,
    python_module_name=python_module_name,
    container_uri=TRAIN_IMAGE,
)

运行自定义训练作业

接下来，您可以通过调用`run`方法来运行自定义作业以开始训练作业。

**注意：** 使用 Vertex AI SDK for Python 提交训练作业时，它会创建一个[训练流水线](https://cloud.google.com/vertex-ai/docs/training/create-training-pipeline)，在 Vertex AI 训练服务上启动自定义作业。

In [ ]:
job.run(
    replica_count=1,
    machine_type=MACHINE_TYPE,
    base_output_dir=base_output_dir,
    tensorboard=TENSORBOARD_INSTANCE_NAME,
    service_account=SERVICE_ACCOUNT,
    args=training_args,
)

查看TensorBoard分析面板

当自定义作业状态切换为运行时，您可以通过Google Cloud控制台上的自定义作业页面或实验页面访问Vertex AI TensorBoard分析面板。

Google Cloud指南[使用Profiler对模型训练性能进行分析](https://cloud.google.com/vertex-ai/docs/training/tensorboard-profiler) 提供了详细的说明来访问Vertex AI TensorBoard分析面板并捕获分析会话。

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = False

job.delete()
tensorboard.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI